In [ ]:
!pip install transformers
!pip gdown

ERROR: unknown command "gdown" - maybe you meant "download"


In [ ]:
import gdown

url = 'https://drive.google.com/uc?id=11ZZ5iO9HqahU_T3JP5DtPZi0uJI6eQEj'
output = 'kogpt_comment_reply_model.pt'  # 원하는 파일 이름과 확장자로 변경
gdown.download(url, output, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=11ZZ5iO9HqahU_T3JP5DtPZi0uJI6eQEj
From (redirected): https://drive.google.com/uc?id=11ZZ5iO9HqahU_T3JP5DtPZi0uJI6eQEj&confirm=t&uuid=9b19120c-52f4-45b3-b74e-b978873f27f5
To: /content/kogpt_comment_reply_model.pt
100%|██████████| 501M/501M [00:05<00:00, 93.2MB/s]


'kogpt_comment_reply_model.pt'

In [ ]:
# 필요한 라이브러리를 임포트합니다.
import torch
from transformers import GPT2Config, GPT2LMHeadModel, PreTrainedTokenizerFast

# 대화에서 사용할 특별 토큰들을 정의합니다.
Q_TKN = "<usr>" # 사용자 입력을 나타내는 토큰
A_TKN = "<sys>" # 시스템의 응답을 나타내는 토큰
BOS = '</s>'    # 문장의 시작을 나타내는 토큰
EOS = '</s>'    # 문장의 종료를 나타내는 토큰
MASK = '<unused0>'  # 마스크 토큰
SENT = '<unused1>'  # 문장 분류를 위한 토큰
PAD = '<pad>'   # 패딩 토큰

# 사전 훈련된 토크나이저를 로드합니다. 토크나이저는 텍스트를 모델이 처리할 수 있는 형태로 변환합니다.
koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                    bos_token=BOS, eos_token=EOS, unk_token='<unk>',
                    pad_token=PAD, mask_token=MASK)

# 사전 훈련된 모델의 구성을 로드합니다.
config = GPT2Config.from_pretrained('skt/kogpt2-base-v2')
# 사전 훈련된 모델을 로드합니다.
model = GPT2LMHeadModel(config)
# 사전 훈련된 모델의 가중치를 로드합니다. CPU를 사용하여 로드합니다.
model.load_state_dict(torch.load('kogpt_comment_reply_model.pt', map_location=torch.device('cpu')))
# 모델을 평가 모드로 설정합니다. 이 모드에서는 모델이 학습 중이 아닐 때의 동작을 합니다.
model.eval()

# 문장 분류를 위한 토큰을 초기화합니다.
sent = "0"

# 모델을 사용하여 답변을 생성하는 함수를 정의합니다.
def dl_model(input_text, max_length=64):

    response = ""   # 응답을 초기화합니다.

    # 최대 길이만큼 반복하여 응답을 생성합니다.
    for _ in range(max_length):

        # 0 Q_TKN + "맛있네요." + SENT + sent + A_TKN + "" > model > response (감)
        # 1 Q_TKN + "맛있네요." + SENT + sent + A_TKN + "감" > model > response (사)
        # 2 Q_TKN + "맛있네요." + SENT + sent + A_TKN + "감사" > model > response (합)
        # 3 Q_TKN + "맛있네요." + SENT + sent + A_TKN + "감사합" > model > response (니)
        # 4 Q_TKN + "맛있네요." + SENT + sent + A_TKN + "감사합니" > model > response (다)
        # 5 Q_TKN + "맛있네요." + SENT + sent + A_TKN + "감사합니다" > model > response (.)
        # 6 Q_TKN + "맛있네요." + SENT + sent + A_TKN + "감사합니다." > model > response (EOS)

        # 입력 텍스트를 모델이 처리할 수 있는 형태로 인코딩합니다.
        input_ids = torch.LongTensor(koGPT2_TOKENIZER.encode(Q_TKN + input_text + SENT + sent + A_TKN + response)).unsqueeze(dim=0)

        # 입력 중 어느 부분이 패딩인지를 나타내는 마스크를 생성합니다.
        attention_mask = input_ids != koGPT2_TOKENIZER.pad_token_id

        # 모델을 사용하여 예측을 수행합니다.
        pred = model(input_ids, attention_mask=attention_mask)
        pred = pred.logits

        # 예측 결과 중 가장 확률이 높은 토큰을 선택합니다.
        gen = koGPT2_TOKENIZER.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().numpy().tolist())[-1]

        # 생성된 토큰이 문장의 끝을 나타내면 반복을 종료합니다.
        if gen == EOS:
            break

        # 생성된 토큰을 응답에 추가합니다. 토큰의 시작 부분에 있는 공백을 제거합니다.
        response += gen.replace("▁", " ")

    return response.strip()

tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
import urllib.request
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix

def generate_reply(input_text):
    output = dl_model(input_text)
    return output

urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/tykimos/tykimos.github.io/master/warehouse/dataset/tarrr_sample_submit.txt",
    filename="tarrr_sample_submit.txt",
)

# 파일을 DataFrame으로 로드
df = pd.read_csv('tarrr_sample_submit.txt', delimiter='\t')

for index, row in df.iterrows():
    comment = row['comment']
    reply = generate_reply(comment)

    print(f"[{index}]")
    print("comment : ", comment)
    print("reply : ", reply)
    print("---------------")

[0]
comment :  완전 내 스타일이에요! 가격도 적당하고 위치도 좋고👌
reply :  완전 내 스타일이에요! 고객님의 의견을 소중히 여기며, 가격과 위치 모두 만족시킬 수 있도록 노력하겠습니다. 다음에도 저희 가게를 찾아주셔서 감사합니다. 다음에도 저희 가게를 찾아주셔서 더 나은 경험을 드리도록 하겠습니다.
---------------
[1]
comment :  맛있긴 한데 양이 너무 적어서 좀... ㅠ
reply :  양에 대한 의견 감사합니다. 양에 대한 의견 감사합니다. 더 많은 양을 준비하도록 하겠습니다.
---------------
[2]
comment :  완전 내 스타일이에요 ㅠㅠ 여기 매장 분위기도 이쁨
reply :  완전 내 스타일이에요! 매장 분위기와 함께 좋은 시간 보내셨다니 기쁩니다! 앞으로도 편안하고 즐거운 매장 환경을 제공하기 위해 노력하겠습니다.
---------------
[3]
comment :  한국의 전통 음식을 잘 표현한 것 같아요. 향토음식의 정취가 느껴져 좋았습니다.
reply :  한국의 전통 음식을 잘 표현해 냈다니 기쁩니다. 감사합니다!
---------------
[4]
comment :  서빙하는 분이 좀 불친절해서 기분이 좀 그랬어요.
reply :  서비스에 불만족하셨다니 진심으로 사과드립니다. 직원 교육을 더 철저히 진행하겠습니다.
---------------
[5]
comment :  여기빵 왜이렇게맛있죠? 대박인데?
reply :  맛에 대한 피드백 감사합니다. 고객님의 피드백을 통해 더욱 향상된 맛을 제공하도록 노력하겠습니다.
---------------
[6]
comment :  맛은 있는데, 가격이 너무 비싸서 가성비는 별로였어요.
reply :  맛에 대한 피드백 감사합니다. 가격과 가치의 균형을 위해 노력하겠습니다.
---------------
[7]
comment :  ㅁㅊ... 여기 진짜 미친듯이 맛있음. 또 올듯ㅋㅋ
reply :  맛에 만족하셨다니 정말 기쁩니다! 앞으로도 변